## Install SAMA


In [ ]:
!pip install git+https://github.com/facebookresearch/segment-anything.git

## Download weights

Model types:
- vit_h (larger) - 2.4 GB
- vit_l - 1.2 GB
- vit_b (smaller) - 358 MB

They are downloaded as pth files, which are PyTorch state dictionaries (contain the state of a model - weights, biases and another params).

In [5]:
from http import HTTPStatus

import requests

MODEL_NAME = "vit_h"
VIT_H_MODEL_URL = "https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth"
VIT_H_MODEL_LOCAL_PATH = "model_vit_h.pth"

response = requests.get(url=VIT_H_MODEL_URL)

if response.status_code == HTTPStatus.OK:
    with open(VIT_H_MODEL_LOCAL_PATH, "wb") as f:
        f.write(response.content)
else:
    print(response.status_code)

## Generating masks

In [7]:
from segment_anything import SamPredictor, sam_model_registry

sam = sam_model_registry[MODEL_NAME](checkpoint=VIT_H_MODEL_LOCAL_PATH)

predictor = SamPredictor(sam_model=sam)

In [10]:
import numpy as np
from PIL import Image


img = Image.open("sunsetx.jpg")
img_data = np.asarray(img)

predictor.set_image(image=img_data)

masks, _, _ = predictor.predict("clouds")

AssertionError: ignored